[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/11_sondaggio_marketing_classe.ipynb)

# Capitolo 5: preferenze nel Digital Marketing della classe DDM

[Analizziamo i risultati del sondaggio](https://alessandrosaccoia.questionpro.com/t/AYs1oZ0FdS) effettuato tra i 27 presenti alla lezione del 17 ottobre 2023

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr

# Read the data
df = pd.read_excel("data/sondaggio_marketing_in_classe.xlsx")

# Display the first few rows of the dataframe
df.head()


# Controllo e imputazione dei valori nulli

In [ ]:
df.isnull().sum()

In [ ]:
numeric_cols_mean = df.mean(numeric_only=True)
df = df.fillna(numeric_cols_mean)

In [ ]:
numeric_cols_mean

In [ ]:
# Controlliamo che non ci siano più valori nulli
df.isnull().sum()

Il dataframe é ripulito

In [ ]:
df.head()

# Visualizzazione delle risposte


In [ ]:
import matplotlib.pyplot as plt

# Supponendo che 'df' sia il tuo DataFrame originale
# Calcola il numero di variabili (N)
N = df.shape[1] - 1  # Escludi la prima colonna se non è numerica

fig, axes = plt.subplots(1, N, figsize=(12, 12), sharey=True)  # Imposta il layout dei subplot

# Ciclo per creare boxplot per ogni variabile
for i, col in enumerate(df.columns[1:]):
    df[[col]].boxplot(ax=axes[i])
    plt.setp(axes[i].xaxis.get_majorticklabels(), rotation=90)  # Ruota le etichette sull'asse x

plt.tight_layout()
plt.show()


# Standardizzazione delle variabili

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.iloc[:, 1:])

# Creazione della matrice di distanze e dendrogramma

In [ ]:
# Clustering gerarchico
Z = linkage(df_scaled, method='ward', metric='euclidean')
Z

In [ ]:
# Visualizza il dendrogramma
plt.figure(figsize=(12, 6))
dendrogram(Z, p=10, truncate_mode='level')
plt.xlabel('Nodi')
plt.ylabel('Distanza')
plt.show()

# Decisione dell'altezza di "taglio"

Dopo aver esaminato il dendrogramma, é possibile valre quali e quanti gruppi tenere. Tagliamo a 8 per avere tre cluster

In [ ]:
threshold = 8

In [ ]:
# Effettua il clustering in base all'altezza di taglio scelta
clusters = fcluster(Z, t=threshold, criterion='distance')
print(np.unique(clusters))
print(clusters)
# Aggiungi la colonna dei cluster al DataFrame originale (non a quello normalizzato, in tal modo )
df['Cluster'] = clusters

## Numerositá e proporzione dei cluster

In [ ]:
cluster_counts = df['Cluster'].value_counts()
proportions = cluster_counts / df.shape[0]

print(cluster_counts)
print(proportions)

In [ ]:
# Calculate centroids for each cluster
cluster_centroids = df.groupby('Cluster').mean().reset_index()
print(cluster_centroids)



## Trasposizione dei cluster per visualizzazione tabella

In [ ]:

# Calculate centroids for each cluster
cluster_centroids = cluster_centroids.round(2)

# Transpose the matrix
cluster_centroids_transposed = cluster_centroids.transpose()

# Correct the column headers after transposition
cluster_centroids_transposed.columns = cluster_centroids_transposed.iloc[0]
cluster_centroids_transposed = cluster_centroids_transposed.drop('Cluster')
cluster_centroids_transposed



In [ ]:
cluster_centroids_transposed.to_excel('data/risultati_clustering_sondaggio.xlsx')

## Visualizzazione con PCA

Notea: I nomi sono visualizzati 10 pixel sopra il punto corrispondente

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Esegui PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Set the style of the plot
sns.set(style="whitegrid")

# Create a scatter plot of the PCA results with colors based on cluster ID
plt.figure(figsize=(10, 6))
scatter = sns.scatterplot(x=df_pca[:, 0], y=df_pca[:, 1], hue=df['Cluster'], palette='tab10', s=80)

# Aggiungi i nomi delle persone come annotazioni
for i, name in enumerate(df['Nome']):
    scatter.annotate(name, (df_pca[i, 0], df_pca[i, 1]), textcoords="offset points", xytext=(0,10), ha='center')

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster', loc='best', labels=['Cluster ' + str(i) for i in range(1, len(np.unique(clusters)) + 1)])
plt.grid(False)
plt.show()


In [ ]:
from scipy.spatial.distance import pdist, squareform
import seaborn as sns

# Calcola le distanze euclidee tra le osservazioni
distances = pdist(df_scaled, metric='euclidean')

# Converti le distanze in una matrice simmetrica
distance_matrix = squareform(distances)

# Visualizza la matrice delle distanze come heatmap con etichette dei nomi delle persone
plt.figure(figsize=(12, 10))
sns.heatmap(distance_matrix, cmap='viridis', xticklabels=df['Nome'], yticklabels=df['Nome'])
plt.title('Matrice delle Distanze Euclidee')
plt.xticks(rotation=90)  # Ruota le etichette sull'asse x per una migliore leggibilità
plt.yticks(rotation=0)
plt.show()


# Trovare le persone più vicine a se stessi

In [ ]:
# Nome da cui calcolare le distanze
whoami = "Alessandro S."

# Trova l'indice dell'osservazione corrispondente al nome specifico. Prende lo [0] perché più righe potrebbero corrispondere al criterio selezionato
idx = df[df['Nome'] == whoami].index[0]

idx # questo é l'indice della riga

In [ ]:
# Estrai le distanze per l'osservazione selezionata
selected_distances = distance_matrix[idx]

# Crea una Serie pandas con i nomi delle persone e le distanze
distance_series = pd.Series(selected_distances, index=df['Nome'], name='Distance')

# Ordina la Serie per distanza crescente (escludendo se stesso)
distance_series = distance_series.drop(whoami).sort_values()

# Visualizza i punti più vicini
print(distance_series)